This jupyter notebook demonstrates dimension reduction summarizes a dataset using its common occuring patterns. In this chapter, you'll learn about the most fundamental of dimension reduction techniques, "Principal Component Analysis" ("PCA"). PCA is often used before supervised learning to improve model performance and generalization. It can also be useful for unsupervised learning. For example, you'll employ a variant of PCA will allow you to cluster Wikipedia articles by their content!

In [1]:
# Import the course packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import scipy.stats 

# Import the course datasets as DataFrames
grains = pd.read_csv('../datasets/grains.csv')
fish = pd.read_csv('../datasets/fish.csv', header=None)
wine = pd.read_csv('../datasets/wine.csv')
eurovision = pd.read_csv('../datasets/eurovision-2016.csv')
stocks = pd.read_csv('../datasets/company-stock-movements-2010-2015-incl.csv', index_col=0)
digits = pd.read_csv('../datasets/lcd-digits.csv', header=None)

# Visualizing the PCA transformation

### Correlated data in nature

You are given an array grains giving the width and length of samples of grain. You suspect that width and length will be correlated. To confirm this, make a scatter plot of width vs length and measure their Pearson correlation.